<a href="https://colab.research.google.com/github/SwaksharDeb/Activity-recognition/blob/master/HAR_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing dependencies

In [49]:
import numpy as np
import matplotlib.pyplot as plt
import math
import csv
import array
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from numpy import savetxt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
import scipy.stats as stats
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam

# Importing the dataset

In [50]:
dataset_user04 = pd.read_csv("final_field_user04_dataset.csv")
dataset_user07 = pd.read_csv("final_field_user07_dataset.csv")
dataset_user08 = pd.read_csv("final_field_user08_dataset.csv")
dataset_user18 = pd.read_csv("final_field_user18_dataset.csv")
dataset_user38 = pd.read_csv("final_field_user38_dataset.csv")
dataset_user51 = pd.read_csv("final_field_user51_dataset.csv")


# Data processing

In [51]:
dataset = pd.concat([dataset_user04, dataset_user07, dataset_user08, dataset_user18, dataset_user38, dataset_user51], ignore_index = True)

In [19]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1779768 entries, 0 to 1779767
Data columns (total 4 columns):
 #   Column  Dtype  
---  ------  -----  
 0   x       float64
 1   y       float64
 2   z       float64
 3   act_id  int64  
dtypes: float64(3), int64(1)
memory usage: 54.3 MB


In [20]:
dataset['act_id'].value_counts()

5     1074348
4      406725
7      108697
3       97370
12      47544
1       22547
6       15586
2        2698
10       2225
9        2028
Name: act_id, dtype: int64

## Balancing the dataset

In [78]:
act_id_1 = dataset[dataset['act_id']==1].head(5028).copy()
act_id_2 = dataset[dataset['act_id']==2].head(5028).copy()
act_id_3 = dataset[dataset['act_id']==3].head(5028).copy()
act_id_4 = dataset[dataset['act_id']==4].head(5028).copy()
act_id_5 = dataset[dataset['act_id']==5].head(5028).copy()
act_id_6 = dataset[dataset['act_id']==6].head(5028).copy()
act_id_7 = dataset[dataset['act_id']==7].head(5028).copy()
act_id_8 = dataset[dataset['act_id']==8].head(5028).copy()
act_id_9 = dataset[dataset['act_id']==9].head(2028).copy()
act_id_10 = dataset[dataset['act_id']==10].head(2225).copy()
act_id_11 = dataset[dataset['act_id']==11].head(2028).copy()
act_id_12 = dataset[dataset['act_id']==12].head(2028).copy()

In [79]:
balanced_data = pd.DataFrame()
balanced_data = balanced_data.append([act_id_1, act_id_2, act_id_3, act_id_4, act_id_5, act_id_6, act_id_7, act_id_8, act_id_9, act_id_10, act_id_11, act_id_12])
balanced_data.reset_index(drop=True, inplace = True)
balanced_data.shape

(39147, 4)

In [80]:
balanced_data.head()

,x,y,z,act_id
0,-0.153,6.397,5.773,1
1,-0.153,6.397,5.773,1
2,-2.068,9.079,5.823,1
3,-4.980,7.891,0.036,1
4,-4.980,7.891,0.036,1


In [82]:
act_id_12.head()

,x,y,z,act_id
98016,-1.494,7.508,7.098,12
98017,-1.494,7.508,7.098,12
98018,-1.494,7.508,7.098,12
98019,-1.494,7.508,7.098,12
98020,-0.996,10.496,2.565,12


In [83]:
balanced_data['act_id'].value_counts()

7     5028
6     5028
5     5028
4     5028
3     5028
1     5028
2     2698
10    2225
12    2028
9     2028
Name: act_id, dtype: int64

## Seperating features and label

In [84]:

X = balanced_data.iloc[:, [0, 1, 2]].values
Y = balanced_data.iloc[:, 3].values

Y = Y - 1

## Feature scaling

In [85]:
sc = StandardScaler()
X_scaled = sc.fit_transform(X)

In [86]:
X_scaled.shape[0]

39147

In [87]:
X_scaled

array([[-0.06635294, -0.47275689,  1.71808463],
       [-0.06635294, -0.47275689,  1.71808463],
       [-0.50618248,  0.55483225,  1.73388061],
       ...,
       [ 2.10661166, -3.01146146, -0.21249948],
       [ 2.09788397, -3.01146146, -0.21249948],
       [ 2.09788397, -2.9823426 , -0.21186765]])

## Making the frame

In [88]:
Fs = 20
frame_size = Fs*4 # 80
hop_size = Fs*2 # 40

def get_frames(X_scaled, frame_size, hop_size):

    N_FEATURES = 3

    frames = []
    labels = []
    for i in range(0, X_scaled.shape[0] - frame_size, hop_size):
        x = X_scaled[i: i + frame_size, 0]
        y = X_scaled[i: i + frame_size, 1]
        z = X_scaled[i: i + frame_size, 2]

        # Retrieve the most often used label in this segment
        label = stats.mode(Y[i: i + frame_size])[0][0]
        frames.append([x, y, z])
        labels.append(label)

    # Bring the segments into a better shape
    frames = np.asarray(frames).reshape(-1, frame_size, N_FEATURES)
    labels = np.asarray(labels)

    return frames, labels

X_, y_ = get_frames(X_scaled, frame_size, hop_size)

In [89]:
X_.shape, y_.shape

((977, 80, 3), (977,))

## Train-Test split

In [90]:
X_train, X_test, y_train, y_test = train_test_split(X_, y_, test_size = 0.2, random_state = 0, stratify = y_)

In [91]:
X_train.shape, X_test.shape

((781, 80, 3), (196, 80, 3))

In [92]:
X_train[0].shape, X_test[0].shape

((80, 3), (80, 3))

In [93]:
X_train = X_train.reshape(-1, frame_size, 3, 1)
X_test = X_test.reshape(-1, frame_size, 3, 1)

In [94]:
X_train[0].shape, X_test[0].shape

((80, 3, 1), (80, 3, 1))

In [95]:
y_train.shape, y_test.shape

((781,), (196,))

In [96]:
y_train[0:10]

array([ 0,  3,  4, 11,  2,  9,  4,  9,  6,  6])

# CNN Model

In [97]:
model = Sequential()
model.add(Conv2D(16, (2, 2), activation = 'relu', padding = 'same', input_shape = X_train[0].shape))
model.add(Dropout(0.1))

model.add(Conv2D(32, (2, 2), activation='relu', padding= 'same'))
model.add(Dropout(0.2))

#model.add(Conv2D(64, (2, 2), activation='relu', padding='same'))
#model.add(Dropout(0.2))

model.add(Flatten())

#model.add(Dense(128, activation = 'relu'))
#model.add(Dropout(0.2))

model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(12, activation='softmax'))

In [98]:
model.compile(optimizer=Adam(learning_rate = 0.001), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [99]:
history = model.fit(X_train, y_train, epochs = 50, validation_data= (X_test, y_test), verbose=1)

Epoch 1/50
25/25 [==============================] - 1s 24ms/step - loss: 2.1322 - accuracy: 0.2791 - val_loss: 1.6287 - val_accuracy: 0.4286
Epoch 2/50
25/25 [==============================] - 1s 20ms/step - loss: 1.8054 - accuracy: 0.3777 - val_loss: 1.5515 - val_accuracy: 0.4643
Epoch 3/50
25/25 [==============================] - 0s 20ms/step - loss: 1.7051 - accuracy: 0.4008 - val_loss: 1.4779 - val_accuracy: 0.4541
Epoch 4/50
25/25 [==============================] - 0s 20ms/step - loss: 1.6281 - accuracy: 0.4072 - val_loss: 1.4921 - val_accuracy: 0.4541
Epoch 5/50
25/25 [==============================] - 1s 20ms/step - loss: 1.5293 - accuracy: 0.4571 - val_loss: 1.4809 - val_accuracy: 0.4286
Epoch 6/50
25/25 [==============================] - 0s 20ms/step - loss: 1.4690 - accuracy: 0.4571 - val_loss: 1.4702 - val_accuracy: 0.4235
Epoch 7/50
25/25 [==============================] - 1s 20ms/step - loss: 1.3794 - accuracy: 0.5058 - val_loss: 1.4897 - val_accuracy: 0.4643
Epoch 8/50
25